In [ ]:
%matplotlib inline
import openpifpaf

openpifpaf.show.Canvas.show = True
openpifpaf.show.Canvas.image_min_dpi = 200

In [ ]:
import io
import numpy as np
import PIL
import requests
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import cv2

print('OpenPifPaf version', openpifpaf.__version__)
print('PyTorch version', torch.__version__)

In [ ]:
pil_im = PIL.Image.open(r"/home/hestia/Documents/Experiments/Test/openpifpaf/openpifpaf/old/sample.jpg").convert('RGB')
im = np.asarray(pil_im)

print(im.shape)

with openpifpaf.show.image_canvas(im) as ax:
    pass

In [ ]:
openpifpaf.plugins.coco.CocoKp().head_metas[0]

In [ ]:
print(im.shape)

In [ ]:
predictor = openpifpaf.Predictor(checkpoint='resnet50')
predictions, gt_anns, image_meta = predictor.pil_image(pil_im)

In [ ]:
# print(gt_anns)
# print(image_meta)
for i in range (len(predictions)):
    print(predictions[i].data.shape)

In [ ]:
# visualize the predicted annotations in `predictions`

annotation_painter = openpifpaf.show.AnnotationPainter()
with openpifpaf.show.image_canvas(im) as ax:
    annotation_painter.annotations(ax, predictions)

In [ ]:
# THe prediction in the `predictions` list is a type of `Annotation` it can access the joint coordinates in the `data` attribute. 
# Numpy array that contains the x and y coors and the confidence for every joint:

predictions[0].data

The `Predictor` class can also be created with `json_data=True` and then it will
return JSON serializable dicts and list instead of `Annotation` objects.

The other items that are returned are ground truth annotations (`gt_anns`) which
are not provided for this image and meta information about the image (`image_meta`)
which is useful to understand the transformations that were applied before
passing the image through the neural network. Usually, you don't need `image_meta`
as the inverse transform has already been applied to ground truth and predictions
in the `Predictor` class:

In [ ]:
gt_anns

In [ ]:
image_meta

In [ ]:
openpifpaf.visualizer.Base.set_all_indices(['cif,caf:5:confidence'])
_ = predictor.pil_image(pil_im)

In [ ]:
openpifpaf.visualizer.Base.set_all_indices(['cif,caf:5:regression'])
_ = predictor.pil_image(pil_im)

From the CIF field, a high resolution accumulation (in the code it's called `CifHr`) is generated.
This is also the basis for the seeds. Both are shown below.

In [ ]:
openpifpaf.visualizer.Base.set_all_indices(['cif:5:hr', 'seeds'])
_ = predictor.pil_image(pil_im)

Starting from a seed, the poses are constructed. At every joint position, an occupancy map marks whether a previous pose was already constructed here. This reduces the number of poses that are constructed from multiple seeds for the same person. The final occupancy map is below:

In [ ]:
openpifpaf.visualizer.Base.set_all_indices(['occupancy:5'])
_ = predictor.pil_image(pil_im)

In [ ]:
openpifpaf.decoder.utils.cif_hr.debug_visualizer = openpifpaf.visualizer.CifHr()
openpifpaf.visualizer.CifHr.show = True

In [ ]:
net_cpu, _ = openpifpaf.network.Factory(checkpoint='resnet50').factory()
device = "cuda" if torch.cuda.is_available() else "cpu"
net = net_cpu.to(device)


openpifpaf.decoder.utils.CifSeeds.threshold = 0.5
openpifpaf.decoder.utils.nms.Keypoints.keypoint_threshold = 0.2
openpifpaf.decoder.utils.nms.Keypoints.instance_threshold = 0.2
processor = openpifpaf.decoder.factory([hn.meta for hn in net.head_nets])

In [ ]:
data = openpifpaf.datasets.PilImageList([pil_im])
loader = torch.utils.data.DataLoader(
  data, batch_size=1, pin_memory=True, 
  collate_fn=openpifpaf.datasets.collate_images_anns_meta)

# openpifpaf.show.Canvas.show = True
# annotation_painter = openpifpaf.show.AnnotationPainter()

for images_batch, _, __ in loader:
  predictions = processor.batch(net, images_batch, device=device)[0]
  # with openpifpaf.show.image_canvas(im) as ax:
  #   annotation_painter.annotations(ax, predictions)
print(predictions[0].data)

In [ ]:
openpifpaf.plugins.coco.CocoKp().head_metas[0]
# image_meta

In [ ]:
# init the fields
field = processor.fields_batch(net, images_batch, device=device)[0]

# Create cif instance 
cif = openpifpaf.visualizer.Cif(openpifpaf.plugins.coco.CocoKp().head_metas[0])
cif.predicted(field[0])

#field [0] is pif while field [1] is paf
print(field[0].shape)


In [ ]:
print(field[0][:, 0].shape)
# print(field[0])

# Example give indexes to cif for a specific joint type (0 - 16)
# debug_indices = ['cif:5']
# print(debug_indices)
# debug_indices = [di.partition(':') for di in debug_indices]
# debug_indices = [(di[0], int(di[2])) for di in debug_indices]
# cif.all_indices = debug_indices

# Create the confidence heatmap
confidences = field[0][:, 0]

print(confidences.shape)

img = plt.imread('/home/hestia/Documents/Experiments/Test/openpifpaf/openpifpaf/old/sample.jpg')

for f in range (16):
    imgplot = plt.imshow(img)
    hm = cif.scale_scalar(confidences[f], 8)
    print(hm.shape)
    plt.imshow(hm , alpha=0.9, vmin=0.0, vmax=1.0, cmap='Oranges')
    plt.show()


In [ ]:
# Create a dummy config arg
config = openpifpaf.decoder.FieldConfig()

for images_batch, _, __ in loader:
    images_batch = images_batch.cuda()

    # Get the field
    field = processor.fields_batch(net, images_batch, device=device)[0]

    # Accumulate the field and extract the heatmap
    cif = openpifpaf.decoder.CifHr(config)
    cif.fill(field)
    heatmap = cif.accumulated.copy()


In [ ]:
# Get the field, I'm not sure what it is, but cif.py is using it
field = processor.fields_batch(net, images_batch, device=device)[0]

# Create cif instance 
cif = openpifpaf.visualizer.Cif(openpifpaf.plugins.coco.CocoKp().head_metas[0])
cif.predicted(field[0])

# Create the heatmap
confidences = field[0][:, 0]

img = plt.imread('/home/hestia/Documents/Experiments/Test/openpifpaf/openpifpaf/old/sample.jpg')

cif_hr = openpifpaf.decoder.utils.CifHr()
cif_hr.fill(field,cif)
hr_heatmap = cif_hr.accumulate()

print(hr_heatmap.shape)

# for f in range (16):
#     imgplot = plt.imshow(img)
#     hm = cif.scale_scalar(confidences[f], 8)
#     print(hm.shape)
#     plt.imshow(hm , alpha=0.9, vmin=0.0, vmax=1.0, cmap='Oranges')
#     plt.show()
